In [30]:
import pandas as pd
import numpy as np

import re
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hussien\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
df = pd.read_csv('CompanyReviews.csv')
df

,Username,Location,Date,Rating,Company,Review
0,Unknown,GB,2 days agoInvited,5,gymshark,"PerfectShipping was so fast, really good quali..."
1,Sorina Irmak,TR,"Apr 24, 2024",4,gymshark,I was unable to apply my discount code… I was ...
2,Tamara Rockall,GB,6 days agoInvited,5,gymshark,Fantastic as alwaysFantastic as always. There ...
3,Lauren,GB,"Apr 15, 2024Invited",5,gymshark,Quick delivery and high quality The website is...
4,Malcolm Rait,GB,4 days agoInvited,5,gymshark,Quality gym clothing easy to purchase…Quality ...
...,...,...,...,...,...,...
43083,Instagram innerpeace_queen,US,"Feb 22, 2022Invited",5,shein,"Great stuffGreat stuff, literally the best!Dat..."
43084,Christina,US,"Feb 22, 2022Invited",5,shein,ADD TO CART!!!!Quick shipping and the workout ...
43085,Audrey,US,"Feb 22, 2022Invited",5,shein,All of my clothes were amazing!Date of experie...
43086,Michaela,US,"Feb 22, 2022Invited",5,shein,@mdot2shoti love sheinnnn y’all pants be off t...


In [101]:
df.isna().sum()

Rating    0
Review    0
dtype: int64

In [67]:
# Regular expression pattern to match "(Date of experience: [date])"
pattern = r'Date of experience: [A-Za-z]+\s\d{1,2},\s\d{4}'

# Function to remove the pattern
def remove_date(text):
    return re.sub(pattern, '', text).strip()

# Apply the function to the text column
df['Review'] = df['Review'].apply(remove_date)

In [91]:
import nltk
from wordsegment import load, segment

# Load the wordsegment data
load()

def split_stuck_words(text):
    # Use wordsegment to split any concatenated words in the text
    words = text.split()
    split_words = []
    for word in words:
        split_words.extend(segment(word))
    return ' '.join(split_words)

def remove_repetition(text):
    # First, split any stuck-together repeated words
    text = split_stuck_words(text)
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    
    # Initialize an empty list to hold the cleaned words
    cleaned_words = []
    
    # Initialize a set to track seen sequences
    seen_sequences = set()
    
    # Iterate over words to detect and remove repetitions
    i = 0
    while i < len(words):
        # Assume no sequence found initially
        found_sequence = False
        
        # Consider sequences of length up to 5 words
        for j in range(5, 0, -1):
            if i + j <= len(words):
                sequence = ' '.join(words[i:i+j])
                # Check if the sequence is already seen
                if sequence in seen_sequences:
                    # If a repeated sequence is found, skip ahead by the length of the sequence
                    i += j
                    found_sequence = True
                    break
                else:
                    # Add the sequence to seen sequences
                    seen_sequences.add(sequence)
        
        if not found_sequence:
            # If no sequence was found, add the current word and move to the next word
            cleaned_words.append(words[i])
            i += 1
    
    # Reconstruct the cleaned text
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

# Example usage
df['Review'] = df['Review'].apply(remove_repetition)  

In [95]:
df["Review"].sample(10)

16808                          great service fast delivery
9314     attention detail evident well care packaged items
1567                                        good great job
17788              fast shipping product quality expecting
42124    excellent product good quality fair price fits...
20424                                  quick easy flawless
3261     website alright bought son really clothing thi...
23312                                          fast smooth
26235    2 day shipping become fib today another exampl...
1322     poor quality cancelled items item ordered refu...
Name: Review, dtype: object

In [68]:
df = df.drop(['Username', 'Location', 'Date', 'Company'], axis = 1)
df.sample(5).head()

,Rating,Review
21462,5,"They were excellentThey were excellent, and a ..."
25202,5,Always great service from AmazonNever had a ba...
28501,2,Apple Was Once Superior at Customer ServiceApp...
1338,5,"UK's finestGreat sales, great kit, from one of..."
30262,5,I love every item I orderedI love every item I...


In [69]:
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Hussien\AppData\Local\Temp\ipykernel_32784\3341848437.py:4: SyntaxWarning: invalid escape sequence '\S'
  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


In [70]:
def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

def map_rating_to_sentiment(rating):
    if rating in [1, 2]:
        return 'Negative'
    elif rating == 3:
        return 'Neutral'
    elif rating in [4, 5]:
        return 'Positive'

In [71]:
df["Review"] = df["Review"].apply(lambda x: preprocess(x))
df['Rating'] = df['Rating'].apply(map_rating_to_sentiment)
df

,Rating,Review
0,Positive,perfectshipping fast really good quality usual
1,Positive,unable apply discount code unable apply discou...
2,Positive,fantastic alwaysfantastic always problem order...
3,Positive,quick delivery high quality website easy use p...
4,Positive,quality gym clothing easy purchase quality gym...
...,...,...
43083,Positive,great stuffgreat stuff literally best
43084,Positive,add cart quick shipping workout clothes amazin...
43085,Positive,clothes amazing
43086,Positive,love sheinnnn pants though sized like anywayss...


In [96]:
df["Rating"].value_counts()

Rating
Positive    31341
Negative    10453
Neutral      1294
Name: count, dtype: int64

In [99]:
df.Review[2]

'fantastic always problem order gym shark rectified following day'

In [100]:
df.to_csv('CleanedReviews.csv', index=False)